### Curso de Python para Finanças Quantitativas

#### Aula 7 - Coletando dados da Investing.com com Python para ações e criptomoedas
#### Autor: Leandro Guerra - Outspoken Market
#### Download em: https://www.outspokenmarket.com/pythonfinancasquantitativas.html

In [1]:
!pip3 install investpy

In [2]:
import investpy as inv

In [3]:
import plotly.graph_objects as go

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
matplotlib.rcParams['figure.figsize'] = (16,8)

In [ ]:
plt.style.use("fivethirtyeight")
# https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html

#### Primeiro vamos entender a lib de um modo geral: sempre leia a documentação 

https://investpy.readthedocs.io

investpy - Financial Data Extraction from Investing.com with Python developed by Alvaro Bartolome del Canto

In [ ]:
# Depois, comecemos as análises

tudo_brasil = inv.get_stocks_overview(country = "Brazil"
                        , as_json = False
                        , n_results = 100)

tudo_brasil

In [ ]:
# Agora, vamos entender algumas funções especiais

ticker = "PETR4" # a lib não aceita um vetor com mais ações como entrada

In [ ]:
# Descrição da empresa

petro_desc = inv.get_stock_company_profile(stock = ticker
                                           , country = "Brazil")

petro_desc

In [ ]:
# Informações de receita

petro_receita = inv.get_stock_financial_summary(stock = ticker
                                                , country = "Brazil"
                                                , summary_type = "income_statement"
                                                , period = "quarterly")

petro_receita

In [ ]:
# Visualização

plt.figure(figsize=(12.5,4.5))
plt.plot(petro_receita["Net Income"], label = "PETR4", linewidth = 2, color = "blue")
plt.title("Receita Total: PETR4 - Investing.com \n Python para finanças Outspoken Market")
plt.xlabel("Últimos 4 trimestres")
plt.ylabel("Receita em $")
plt.legend(loc = "lower right")
plt.show()

In [ ]:
# Mas quais outras informações eu posso pegar?
# Consulte sempre a documentação. Quando ela não estiver tão clara
# tente também a função help(nome_função)

help(inv.get_stock_financial_summary)

In [ ]:
# Para informações adicionais da ação

petro = inv.get_stock_information(stock = ticker
                                  , country = "Brazil"
                                  , as_json = False).transpose()

petro

In [ ]:
# Dados históricos da cotação

inicio = "01/01/2020"
fim = "22/11/2021"

petr4 = inv.get_stock_historical_data(stock = ticker
                                      , country = "Brazil"
                                      , from_date = inicio
                                      , to_date = fim)
petr4

In [ ]:
# Visualização

plt.figure(figsize = (12.5,4.5))
plt.plot(petr4["Close"], label = "PETR4", linewidth = 2, color = "blue")
plt.title("PETR4 - Investing.com \n Python para finanças Outspoken Market")
plt.xlabel("2020 à 2021")
plt.ylabel("Cotação (R$)")
plt.legend(loc = "lower right")
plt.show()

In [ ]:
# Mas, e se eu quiser os indicadores técnicos?

petro_ind = inv.search_quotes(text = ticker
                              , products=["stocks"]
                              , countries=["brazil"]
                              , n_results = 1)
print(petro_ind)

In [ ]:
# Use o método retrieve_technical_indicators

petro_ind_d1 = petro_ind.retrieve_technical_indicators(interval = "daily")
petro_ind_d1

In [ ]:
petro_ind_semanal = petro_ind.retrieve_technical_indicators(interval = "weekly")
petro_ind_semanal

In [ ]:
petro_ind_mensal = petro_ind.retrieve_technical_indicators(interval = "monthly")
petro_ind_mensal

In [ ]:
# Vamos combinar os valores dos diferentes timeframes em uma única tabela


petro_at = pd.DataFrame([petro_ind_d1["indicator"], petro_ind_d1["value"]
                         , petro_ind_semanal["value"], petro_ind_mensal["value"]]).transpose()

petro_at

In [ ]:
petro_at.set_axis(["Indicador", "D1", "Semanal", "Mensal"], axis = 1, inplace = True)
petro_at.set_index(petro_at["Indicador"], drop = True, inplace = True)
petro_at

In [ ]:
# Selecionando apenas os indicadores de momentum

petro_momentum = petro_at.iloc[[0, 4, 5, 6], :].copy()
petro_momentum

In [ ]:
petro_momentum.plot(kind = "bar"
              , title = "Indicadores de Momentum PETR4 \n Python para finanças Outspoken Market"
              , figsize = (12, 7))
plt.ylabel("Valor do Indicador")
plt.ylim(-120, 100)
plt.legend(loc = "upper right");

### Para as criptomoedas, temos funções específicas 

In [ ]:
inv.search_cryptos(by = "name"
                   , value = "bitcoin")

# Repare que o método retorna tudo o que tem BTC no nome
# A lib não permite alterar a moeda

In [ ]:
btc = inv.get_crypto_historical_data(crypto = "Bitcoin"
                                     , from_date = inicio
                                     , to_date = fim)
btc

In [ ]:
# Visualização

plt.figure(figsize = (12.5,4.5))
plt.plot(btc["Close"], label = "BTC", linewidth = 2, color = "blue")
plt.title("Bitcoin - Investing.com \n Python para finanças Outspoken Market")
plt.xlabel("2020 à 2021")
plt.ylabel("Cotação (US$)")
plt.legend(loc = "lower right")
plt.show()

In [ ]:
# E como recuperar informações de outras exchanges?

busca_cripto = inv.search_quotes(text = "Bitcoin"
                                 , products = ["cryptos"]
                                 , n_results = 100)

In [ ]:
# Parece um dicionário ou um json, mas nao é

type(busca_cripto)

In [ ]:
for item in busca_cripto:
    print(item)

In [ ]:
type(busca_cripto[0])

In [ ]:
# Então como eu leio isso?

busca_cripto[0].exchange

In [ ]:
data = []
for item in busca_cripto:
    data.append([item.name, item.symbol, item.exchange])
    
busca_cripto_df = pd.DataFrame(data, columns = ["Nome", "Simbolo", "Exchange"])

busca_cripto_df.head()

In [ ]:
# O códico acima serve para estrutuar e saber o que existe com o nome "Bitcoin"
# Útil para fazer a busca depois, como o exemplo abaixo

inv.get_crypto_historical_data(crypto = "Monetha"
                                     , from_date = inicio
                                     , to_date = fim)

In [ ]:
monetha = inv.get_crypto_historical_data(crypto = "Monetha"
                                     , from_date = inicio
                                     , to_date = fim)

In [ ]:
# Visualização

plt.figure(figsize = (12.5,4.5))
plt.plot(monetha["Close"], label = "Monetha", linewidth = 2, color = "blue")
plt.title("Monetha - Investing.com \n Python para finanças Outspoken Market")
plt.xlabel("2020 à 2021")
plt.ylabel("Cotação (US$)")
plt.legend(loc = "lower right")
plt.show()

### Calendário Economico

In [ ]:
inv.economic_calendar()

In [ ]:
# Filtrando para 2021 apenas os eventos importantes

agenda21 = inv.economic_calendar(from_date = "01/01/2021"
                                 , to_date = fim)

In [ ]:
agenda21.query("importance == 'high'")

In [ ]:
agenda21.query("importance == 'high'")["event"].unique()

In [ ]:
agenda21.query('importance == "high" and event.str.contains("ADP Nonfarm Employment Change")', engine = "python")

In [ ]:
adp_21 = agenda21.query('importance == "high" and event.str.contains("ADP Nonfarm Employment Change")'
                        , engine = "python")
adp_21.set_index(adp_21["date"], drop = True, inplace = True)
adp_21 = adp_21.iloc[:, [7, 8, 9]]
adp_21.replace("K", "", regex = True, inplace = True)
adp_21["actual"] = pd.to_numeric(adp_21["actual"])
adp_21["forecast"] = pd.to_numeric(adp_21["forecast"])
adp_21["previous"] = pd.to_numeric(adp_21["previous"])
adp_21

In [ ]:
adp_21.plot(kind = "bar"
              , title = "ADP Nonfarm Employment Change \n Python para finanças Outspoken Market"
              , figsize = (12, 7))
plt.ylabel("ADP")
plt.xlabel("Medição")
plt.ylim(-150, 1000)
plt.legend(loc = "upper right");